# RAG Simple (Bee)
- By Marcelo Rovai @28Sept24

## Import the Libraries

In [19]:
import ollama
import chromadb
import time

### Define the models

In [21]:
#start_time = time.perf_counter()  # Start timing
EMB_MODEL = "nomic-embed-text" #"mxbai-embed-large" #"all-minilm" 
MODEL = "llama3.2:3b"

### Enter with the Document

In [4]:
documents = [
    "Bee-keeping, also known as apiculture, involves the maintenance of bee colonies, typically in hives, by humans.",
    "The most commonly kept species of bees is the European honey bee (Apis mellifera).",
    "Bee-keeping dates back to at least 4,500 years ago, with evidence of ancient Egyptians practicing it.",
    "A beekeeper's primary role is to manage hives to ensure the health of the bee colony and maximize honey production.",
    "Honey bees are social insects, living in colonies with a single queen, numerous worker bees, and drones.",
    "The queen bee can lay up to 2,000 eggs per day during peak seasons.",
    "Worker bees are female and perform all the tasks in the hive except for reproduction.",
    "Drones are male bees whose primary role is to mate with a queen from another hive.",
    "Honey bees communicate with each other through the 'waggle dance,' which indicates the direction and distance to food sources.",
    "Bees produce honey from the nectar they collect from flowers, which they store in the hive for food during winter.",
    "Bees also produce beeswax, which they use to build the honeycomb structure in the hive.",
    "Propolis, another bee product, is a resin-like substance collected from tree buds and used to seal gaps in the hive.",
    "Bees play a crucial role in pollination, which is essential for the reproduction of many plants and crops.",
    "A typical bee colony can contain between 20,000 and 80,000 bees.",
    "Bee-keeping can be done for various purposes, including honey production, pollination services, and the sale of bees and related products.",
    "Beekeepers must inspect their hives regularly to check for diseases, pests, and the overall health of the colony.",
    "Common pests and diseases that affect bees include varroa mites, hive beetles, and foulbrood.",
    "Bee-keeping requires protective clothing and equipment, such as a bee suit, gloves, and a smoker to calm the bees.",
    "Sustainable bee-keeping practices are important for maintaining healthy bee populations and ecosystems.",
    "Beekeeping can be a hobby, a part-time occupation, or a full-time profession, depending on the scale and intent of the beekeeper.",
    "Almost all the honey we consume comes from western honey bees (Apis mellifera), a hybrid of European and African species.", 
    "There are another 20,000 different bee species in the world.",  
    "Brazil alone has more than 300 different bee species, and the vast majority, unlike western honey bees, don’t sting.", 
    "Reports written in 1577 by Hans Staden, mention three native bees used by indigenous people in Brazil.",
    "The indigenous people in Brazil used bees for medicine and food purposes",
    "From Hans Staden report: probable species: mandaçaia (Melipona quadrifasciata), mandaguari (Scaptotrigona postica) and jataí-amarela (Tetragonisca angustula)."
]

### Create our vector embedding database  `bee_facts` and store the document in it:

In [5]:
client = chromadb.Client()
collection = client.create_collection(name="bee_facts")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model=EMB_MODEL, prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [9]:
len(embedding)

768

In [35]:
# print(embedding)

In [10]:
len(documents)

26

In [13]:
documents[0]

'Bee-keeping, also known as apiculture, involves the maintenance of bee colonies, typically in hives, by humans.'

### Retrieve the most relevant document given a prompt:

In [14]:
# Prompt
prompt = "How many bees are in a colony? Who lays eggs and how much? How about \
          common pests and diseases?"

In [15]:
# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model=EMB_MODEL
)

In [16]:
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=5
)
data = results['documents']

In [17]:
data

[['A typical bee colony can contain between 20,000 and 80,000 bees.',
  'Beekeepers must inspect their hives regularly to check for diseases, pests, and the overall health of the colony.',
  'Common pests and diseases that affect bees include varroa mites, hive beetles, and foulbrood.',
  'Honey bees are social insects, living in colonies with a single queen, numerous worker bees, and drones.',
  'The queen bee can lay up to 2,000 eggs per day during peak seasons.']]

### Using the prompt and the document retrieved to generate an answer!

In [22]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model=MODEL,
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}",
  options={
    "temperature": 0.0,
    "top_k":10,
    "top_p":0.5                          }
)

In [24]:
print(output['response'])

Based on the provided data, here are the answers to your questions:

1. How many bees are in a colony?
A typical bee colony can contain between 20,000 and 80,000 bees.

2. Who lays eggs and how much?
The queen bee lays up to 2,000 eggs per day during peak seasons.

3. What about common pests and diseases?
Common pests and diseases that affect bees include varroa mites, hive beetles, and foulbrood.


### More Examples

In [27]:
def rag_bees(prompt):
    start_time = time.perf_counter()  # Start timing
    
    # generate an embedding for the prompt and retrieve the most relevant doc (data)
    response = ollama.embeddings(
      prompt=prompt,
      model=EMB_MODEL
    )
    
    results = collection.query(
      query_embeddings=[response["embedding"]],
      n_results=5
    )
    data = results['documents']
    
    # generate a response combining the prompt and data retrieved
    output = ollama.generate(
      model=MODEL,
      prompt=f"Using this data: {data}. Respond to this prompt: {prompt}",
      options={
        "temperature": 0.0,
        "top_k":10,
        "top_p":0.5                          }
    )
    
    print(output['response'])
    
    end_time = time.perf_counter()  # End timing
    elapsed_time = round((end_time - start_time), 1)  # Calculate elapsed time
    
    print(f"\n [INFO] ==> The code for model: {MODEL}, took {elapsed_time}s to generate the answer.\n")

In [32]:
def rag_bees(prompt, n_results=5, temp=0.0, top_k=10, top_p=0.5):
    start_time = time.perf_counter()  # Start timing
    
    # generate an embedding for the prompt and retrieve the data 
    response = ollama.embeddings(
      prompt=prompt,
      model=EMB_MODEL
    )
    
    results = collection.query(
      query_embeddings=[response["embedding"]],
      n_results=n_results
    )
    data = results['documents']
    
    # generate a response combining the prompt and data retrieved
    output = ollama.generate(
      model=MODEL,
      prompt=f"Using this data: {data}. Respond to this prompt: {prompt}",
      options={
        "temperature": temp,
        "top_k": top_k,
        "top_p": top_p                          }
    )
    
    print(output['response'])
    
    end_time = time.perf_counter()  # End timing
    elapsed_time = round((end_time - start_time), 1)  # Calculate elapsed time
    
    print(f"\n [INFO] ==> The code for model: {MODEL}, took {elapsed_time}s \
          to generate the answer.\n")

In [28]:
prompt = "How many bees are in a colony? Who lays eggs and how much? How about \
          common pests and diseases?"
rag_bees(prompt)

Based on the provided data, here are the answers to your questions:

1. How many bees are in a colony?
A typical bee colony can contain between 20,000 and 80,000 bees.

2. Who lays eggs and how much?
The queen bee lays up to 2,000 eggs per day during peak seasons.

3. What about common pests and diseases?
Common pests and diseases that affect bees include varroa mites, hive beetles, and foulbrood.

 [INFO] ==> The code for model: llama3.2:3b, took 24.1s to generate the answer.



In [29]:
prompt = "Existem abelhas no Brazil?"
rag_bees(prompt)

Sim, existem abelhas no Brasil! De acordo com o relato de Hans Staden, há três espécies de abelhas nativas do Brasil que foram mencionadas: mandaçaia (Melipona quadrifasciata), mandaguari (Scaptotrigona postica) e jataí-amarela (Tetragonisca angustula). Além disso, o Brasil é conhecido por ter mais de 300 espécies diferentes de abelhas, a maioria das quais não é agressiva e não põe veneno.

 [INFO] ==> The code for model: llama3.2:3b, took 54.6s to generate the answer.



In [34]:
prompt = "Existem abelhas no Brazil?"
rag_bees(prompt)

Sim, existem abelhas no Brasil! De acordo com o relato de Hans Staden, há três espécies de abelhas nativas do Brasil que foram mencionadas: mandaçaia (Melipona quadrifasciata), mandaguari (Scaptotrigona postica) e jataí-amarela (Tetragonisca angustula). Além disso, o Brasil é conhecido por ter mais de 300 espécies diferentes de abelhas, a maioria das quais não é agressiva e não põe veneno.

 [INFO] ==> The code for model: llama3.2:3b, took 54.1s           to generate the answer.



In [30]:
prompt = "Quantas espécies de abelhas existem abelhas no Brazil?"
rag_bees(prompt)

Existem mais de 300 espécies de abelhas diferentes no Brasil.

 [INFO] ==> The code for model: llama3.2:3b, took 30.0s to generate the answer.



In [31]:
prompt = "Are bees in Brazil?"
rag_bees(prompt)

Yes, bees are found in Brazil. According to the data, Brazil has more than 300 different bee species, and indigenous people in Brazil used bees for medicine and food purposes. Additionally, reports from 1577 mention three native bees used by indigenous people in Brazil.

 [INFO] ==> The code for model: llama3.2:3b, took 22.7s to generate the answer.



In [33]:
prompt = "What bees?"
rag_bees(prompt)

The most commonly kept species of bees is the European honey bee (Apis mellifera).

 [INFO] ==> The code for model: llama3.2:3b, took 32.0s           to generate the answer.

